<a href="https://colab.research.google.com/github/CostrunLarisa/Big-Data/blob/main/YoutubeComments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Citirea datelor

In [4]:
pip install pyspark


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=196982f9f04abf2a54d8949be36989bda6f77b5f623f12b8c6cc56c63945bcf1
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [18]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ytcomments').getOrCreate()
data = spark.read.csv('sample_data/youtube_dataset.csv',inferSchema=True,
                     header=True)
data.printSchema()

root
 |-- Video Name: string (nullable = true)
 |-- Channel Name: string (nullable = true)
 |-- Comment Id: string (nullable = true)
 |-- User Name: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Likes: string (nullable = true)



## Data Preprocessing

Vom elimina coloana "User Name", deoarece nu este un element relevant in analiza noastra.

In [36]:
# Stergem liniile care au coloana de like-uri sau comentariu null

data = data.na.drop(subset=["Likes", "Comment"])
data = data.drop("User Name")
data.printSchema()
data.columns

root
 |-- Video Name: string (nullable = true)
 |-- Channel Name: string (nullable = true)
 |-- Comment Id: string (nullable = true)
 |-- Comment: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Likes: double (nullable = true)



['Video Name', 'Channel Name', 'Comment Id', 'Comment', 'Date', 'Likes']

In [39]:
from pyspark.sql.functions import length

#Adaugam o coloana noua, Comment_Length, pentru a evita procesarea fiecarei linii

data = data.withColumn("Comment_Length", length(data["Comment"]))
data.columns

['Video Name',
 'Channel Name',
 'Comment Id',
 'Comment',
 'Date',
 'Likes',
 'Comment_Length']

## Formatarea pentru MLlib

In [45]:
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.types import ArrayType, DoubleType

data = data.withColumn("Likes", data["Likes"].cast("double"))
assembler = VectorAssembler(inputCols=['Likes', 'Comment_Length'],
                            outputCol='features')

output = assembler.transform(data)
final_data = output.select('features','Video Name')

# Show the DataFrame
final_data.show()

+---------------+--------------------+
|       features|          Video Name|
+---------------+--------------------+
| [98280.0,63.0]|Luis Fonsi - Desp...|
|    [13.0,67.0]|Luis Fonsi - Desp...|
|[370547.0,51.0]|Luis Fonsi - Desp...|
|   [763.0,77.0]|Luis Fonsi - Desp...|
|    [94.0,56.0]|Luis Fonsi - Desp...|
|    [45.0,15.0]|Luis Fonsi - Desp...|
| [36446.0,56.0]|Luis Fonsi - Desp...|
|   [142.0,51.0]|Luis Fonsi - Desp...|
|    [10.0,48.0]|Luis Fonsi - Desp...|
|   [109.0,85.0]|Luis Fonsi - Desp...|
|[321690.0,33.0]|Luis Fonsi - Desp...|
|     [8.0,69.0]|Luis Fonsi - Desp...|
|   [166.0,64.0]|Luis Fonsi - Desp...|
|      [7.0,9.0]|Luis Fonsi - Desp...|
|     [0.0,64.0]|Luis Fonsi - Desp...|
|   [114.0,66.0]|Luis Fonsi - Desp...|
| [10412.0,40.0]|Luis Fonsi - Desp...|
|     [5.0,73.0]|Luis Fonsi - Desp...|
|     [2.0,63.0]|Luis Fonsi - Desp...|
|  [8075.0,55.0]|Luis Fonsi - Desp...|
+---------------+--------------------+
only showing top 20 rows



## Impartirea setului de date

In [46]:
train_yt, test_yt = final_data.randomSplit([0.7,0.3])

## Antrenarea modelului

## Antrenarea modelului dupa ce am selectat numai liniile unde nr. de like-uri > 100

